In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import matplotlib.pyplot as plt

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the aerodynamic data from the .mat file
mat_file_path = '../data/Low-rise with eaves/roof type o/height 3;4/Cp_ts_ROH18_deg068.mat'  # Replace with the correct file path
mat_data = loadmat(mat_file_path)

# Extract relevant data
# Assuming the file contains data like wind pressure coefficients, roof geometry, etc.
pressure_coefficients = mat_data['Wind_pressure_coefficients']  # Replace 'Cp' with the correct key from the .mat file
time_series = mat_data['time']  # Replace with the correct key if available
geometry = mat_data['geometry']  # Replace with the correct key if available

# Create a DataFrame for analysis
df = pd.DataFrame(pressure_coefficients, columns=['Cp_' + str(i) for i in range(pressure_coefficients.shape[1])])
df['time'] = time_series

# Feature Engineering
# Derive aerodynamic properties (e.g., Reynolds number, pressure gradients, etc.)
roof_height = 8.0  # Example value in meters
roof_breadth = 16.0  # Example value in meters
roof_depth = 24.0  # Example value in meters
wind_speed = 10.0  # Example value in m/s
density_air = 1.225  # Air density in kg/m^3
viscosity_air = 1.81e-5  # Air viscosity in Pa·s

# Reynolds number
df['Re'] = (density_air * wind_speed * roof_depth) / viscosity_air

# Add other aerodynamic features (e.g., angles, pressure gradients)
# Example for pressure gradients
df['pressure_gradient'] = df['Cp_0'] - df['Cp_1']

# Prepare dataset
X = df.drop(columns=['Cp_0'])  # Replace 'Cp_0' with the target column
y = df['Cp_0']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
xgb_model.fit(X_train_scaled, y_train)

# PIML model definition
class PIMLModel(nn.Module):
    def __init__(self, input_size):
        super(PIMLModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# PyTorch Dataset
class PressureDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32).to(device)
        self.targets = torch.tensor(targets, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Create PyTorch dataset and dataloaders
train_dataset = PressureDataset(X_train_scaled, y_train.values)
test_dataset = PressureDataset(X_test_scaled, y_test.values)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train PIML model
piml_model = PIMLModel(input_size=X_train_scaled.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(piml_model.parameters(), lr=0.001)

num_epochs = 50
train_losses = []

for epoch in range(num_epochs):
    piml_model.train()
    epoch_loss = 0.0
    for features, targets in train_loader:
        optimizer.zero_grad()
        outputs = piml_model(features)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    train_losses.append(epoch_loss / len(train_loader))
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}")

# Evaluate PIML model
piml_model.eval()
y_pred_p = []
y_true_p = []

with torch.no_grad():
    for features, targets in test_loader:
        outputs = piml_model(features)
        y_pred_p.append(outputs.squeeze().cpu().numpy())
        y_true_p.append(targets.cpu().numpy())

y_pred_p = np.concatenate(y_pred_p)
y_true_p = np.concatenate(y_true_p)

# Visualization
plt.figure(figsize=(8, 6))
plt.scatter(y_true_p, y_pred_p, alpha=0.5, color='blue', edgecolor='k', label='PIML')
plt.scatter(y_test, xgb_model.predict(X_test_scaled), alpha=0.5, color='red', edgecolor='k', label='XGBoost')
plt.xlabel("True Pressure Coefficients")
plt.ylabel("Predicted Pressure Coefficients")
plt.title("True vs Predicted Pressure Coefficients")
plt.grid(True)
plt.legend()
plt.show()


Using device: cuda


KeyError: 'Cp'